In [1]:
from sklearn.cluster import KMeans
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

### This code will produce arbitrary samples of binary deep eutectic solvents, taking the minimum and maximum mole fractions of each component, number of samples desired to make, and number of trials for the k-means clustering (should be much greater than the number of samples desired to produce). This can easily be modified for ternary deep eutectic solvents. 

In [2]:
def Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    
    #The minimum mole fractions of each component.
    lower_bounds = np.array([min_QAS, min_HBD])
    
    #The maximum mole fractions of each component. 
    upper_bounds = np.array([max_QAS, max_HBD])

    #Generating random DES compositions within the design contraints, mole fractions of each composition must = 1. 
    DES_trials = np.random.rand(trials*20,2)
    DES_trials = DES_trials*(upper_bounds-lower_bounds)+lower_bounds

    #Adding mole fractions of each component in the random trial. 
    mole_sum = np.sum(DES_trials, axis=1)
    
    #Divide each component by the sum of all components to obtain compositions whose mole fractions =1. 
    DES_samples = DES_trials/mole_sum[:,None]
    
    #This normalization may still lead to compositions that do not satisfy the constraint 
    
    #isolating compositions that do not meet upper bound contraints
    upper_check = DES_samples>upper_bounds
    
    #isolating compositions that do not meet upper bound contraints
    lower_check = DES_samples<lower_bounds
    
    #Combine all checks, compositions not meeting constraints will be removed
    combined_check = np.append(upper_check, lower_check, axis=1)

    #compositions that have no violations are added to SafeList 
    SafeList = np.any(combined_check, axis=1)

    #compositions violating constraints are added to DeleteList
    DeleteList = ~SafeList

    
    Feasible_DES_samples = DES_samples[DeleteList,:]
    print(" "+str(len(Feasible_DES_samples))+" feasible DES samples generated, clustered into "+str(samples)+" samples")

    #Apply K-means clustering to DES samples
    #Number of Clusters = Final desired samples
    kmeans = KMeans(n_clusters=samples, random_state=0).fit(Feasible_DES_samples)
    
    
    DES_Centroids = kmeans.cluster_centers_

    return(DES_Centroids)



### Generate binary DES with lower mole fractions of 0.2, 0.3 (QAS, HBD) and upper mole fractions of 0.7, 0.8 (QAS, HBD). Produce 96 samples with 192 trials for the clustering.

In [3]:
Binary_DES_Generator(0.2, 0.3, 0.7, 0.8, 96, 192)

 3840 feasible DES samples generated, clustered into 96 samples


array([[0.49439477, 0.50560523],
       [0.32323275, 0.67676725],
       [0.58337013, 0.41662987],
       [0.42258968, 0.57741032],
       [0.24047917, 0.75952083],
       [0.37919338, 0.62080662],
       [0.6203681 , 0.3796319 ],
       [0.53304246, 0.46695754],
       [0.4599526 , 0.5400474 ],
       [0.66913284, 0.33086716],
       [0.28558654, 0.71441346],
       [0.35700248, 0.64299752],
       [0.51628817, 0.48371183],
       [0.39903474, 0.60096526],
       [0.55822511, 0.44177489],
       [0.43435703, 0.56564297],
       [0.64717228, 0.35282772],
       [0.26761597, 0.73238403],
       [0.47781463, 0.52218537],
       [0.60290021, 0.39709979],
       [0.30709483, 0.69290517],
       [0.20870193, 0.79129807],
       [0.33644975, 0.66355025],
       [0.44873705, 0.55126295],
       [0.49934273, 0.50065727],
       [0.25387823, 0.74612177],
       [0.41002978, 0.58997022],
       [0.54834213, 0.45165787],
       [0.36629174, 0.63370826],
       [0.50472879, 0.49527121],
       [0.

### May also save the output as a variable. Note you will get different mole fractions each time so be sure to keep track if you need them.

In [4]:
A = Binary_DES_Generator(0.3, 0.3, 0.7, 0.8, 96, 192)
A 

 3795 feasible DES samples generated, clustered into 96 samples


array([[0.63223675, 0.36776325],
       [0.42622463, 0.57377537],
       [0.52938953, 0.47061047],
       [0.35282144, 0.64717856],
       [0.47468612, 0.52531388],
       [0.58156104, 0.41843896],
       [0.38657453, 0.61342547],
       [0.3133856 , 0.6866144 ],
       [0.45340861, 0.54659139],
       [0.49906404, 0.50093596],
       [0.67197979, 0.32802021],
       [0.53730861, 0.46269139],
       [0.56109314, 0.43890686],
       [0.59433082, 0.40566918],
       [0.40033259, 0.59966741],
       [0.50743034, 0.49256966],
       [0.43876574, 0.56123426],
       [0.61340412, 0.38659588],
       [0.33428997, 0.66571003],
       [0.65662224, 0.34337776],
       [0.36964142, 0.63035858],
       [0.4161699 , 0.5838301 ],
       [0.46263993, 0.53736007],
       [0.5426739 , 0.4573261 ],
       [0.57038449, 0.42961551],
       [0.48564603, 0.51435397],
       [0.51126061, 0.48873939],
       [0.52264881, 0.47735119],
       [0.63949477, 0.36050523],
       [0.37931707, 0.62068293],
       [0.

### The next function will combine the previous function with code to convert mole fractions to a list of lists of volumes that can be directly inputed into code for sample preparation in a pipetting robot (OT-2).

In [5]:
def convert_mole_fractions_to_volumes(stock_QAS, stock_HBD, min_QAS, min_HBD, max_QAS, max_HBD, samples, trials):
    QAS = [] # empty list to append calculated volumes in microL
    HBD = []
    DES_mole_fractions = Binary_DES_Generator(min_QAS, min_HBD, max_QAS, max_HBD, samples, trials)
    for row in DES_mole_fractions: 
        def f(x) :
            y = np.zeros(np.size(x))                                                                           
            y[0] = x[0] + x[1] - 125  #input desired volume                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]
            return y
        x0 = np.array([100.0, 100.0, 100.0])                                        
        x = fsolve(f, x0)
        QAS.append(x[0])
        HBD.append(x[1])
        volumes = [QAS,HBD]

    return(volumes)

In [6]:
convert_mole_fractions_to_volumes(3, 4, 0.3, 0.2, 0.8, 0.7, 96, 192 )

 3840 feasible DES samples generated, clustered into 96 samples


/Users/studentuser/miniconda3/envs/opentrons/lib/python3.6/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


[[92.69493371766282,
  72.33094521379218,
  56.95305405864436,
  78.64482704566198,
  86.0384391888954,
  97.2325943026834,
  64.46466882267671,
  50.704679525351885,
  100.85095853144445,
  82.92908357487762,
  68.4180214905234,
  75.71647744031544,
  88.68603267988425,
  61.770285883614925,
  70.82519804512764,
  94.81966362893026,
  103.0270711025821,
  80.75999620677813,
  54.46887723282683,
  91.49681277143529,
  84.5191762545051,
  59.19170329000313,
  66.83138762137695,
  77.29236089045351,
  74.34435576888711,
  99.20191815101839,
  89.87863795909188,
  48.25868592050593,
  70.04926935641481,
  87.46189145881135,
  63.04646583827234,
  94.14313753663853,
  81.88633037088103,
  58.50187542180108,
  51.94883138469293,
  80.15819306115884,
  65.65232933029087,
  95.95832898000947,
  60.54102118447567,
  85.5014507484597,
  83.773539254748,
  76.50737906271677,
  71.80547859970062,
  49.611862222530576,
  67.3604323117424,
  73.25973459352261,
  79.59354466325182,
  102.22826631539

### Alternatively, you could use this next function if you already have a list you want to convert (as before when we saved the output of the des mole fractions)

In [7]:
def convert_mole_fractions_to_volumes_2(DES_mole_fractions, stock_QAS, stock_HBD):
    QAS = [] # empty list to append calculated volumes in microL
    HBD = []
    for row in DES_mole_fractions: 
        def f(x) :
            y = np.zeros(np.size(x))                                                                           
            y[0] = x[0] + x[1] - 150  #input desired volume                                                                   
            y[1] = ((stock_QAS*x[0])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[0]
            y[2] = ((stock_HBD*x[1])/((stock_QAS*x[0]) + (stock_HBD*x[1]))) - row[1]
            return y
        x0 = np.array([100.0, 100.0, 100.0])                                        
        x = fsolve(f, x0)
        QAS.append(x[0])
        HBD.append(x[1])
        volumes = [QAS,HBD]

    return(volumes)

In [8]:
convert_mole_fractions_to_volumes_2(A, 3, 2)

[[80.10556934813553,
  49.680866390353366,
  64.28243584962607,
  39.98462113301686,
  56.39134145700909,
  72.14087812014523,
  44.37561331084162,
  34.994109968885915,
  53.413587524933654,
  59.86527237463845,
  86.5945812210459,
  65.4538303234753,
  69.01778255797657,
  74.11627835178977,
  46.1981311058439,
  61.07315802251871,
  51.39308228986207,
  77.10615693364998,
  37.62111825917666,
  84.06099797276153,
  42.15867259783958,
  48.32011561721356,
  54.69936249126522,
  66.25175583396803,
  70.42898128575476,
  57.94482818730187,
  61.62886507262562,
  63.29124628030837,
  81.27430904788541,
  43.4219338484639,
  36.292415077009515,
  38.80971580003034,
  67.45064226426645,
  51.94322669781192,
  78.94822765964975,
  34.44684898020133,
  89.47819832673129,
  75.29257473536792,
  47.39819042753271,
  50.594628060931775,
  40.62815236921641,
  70.9972029169704,
  54.240262479968834,
  48.803890035709294,
  59.357225844633106,
  72.78028334673193,
  85.72546405383822,
  58.92213

### This code can be used to produce binary DES from the list of volumes and cocnentrated stock solutions of your components. Capable of switching between two different pipettes, but modify as necessary. Code is originally from Newcastle IGEM at Newcastle University, github can be found here https://github.com/jbird1223/Newcastle-iGEM

In [11]:
from opentrons import labware, instruments, robot
from sqlite3 import IntegrityError
robot.reset() #remove this when uplaoding code to robot

#####################################################################################################################################################
#####################################################################################################################################################		

# Import Labware

tiprack_300 = labware.load("opentrons-tiprack-300ul", '1')      
tiprack_1000 = labware.load("tiprack-1000ul", '4')                
Stock1 = labware.load("opentrons-tuberack-15ml", '2')                                           
wellplate_96 = labware.load("96-flat", '3')
trash = robot.fixed_trash

#####################################################################################################################################################
#####################################################################################################################################################

# Import Pipettes

P1000 = instruments.P1000_Single(
    mount = 'right',
    tip_racks = [tiprack_1000],
    trash_container = trash
)

P300 = instruments.P300_Single(
    mount='left',
    tip_racks=[tiprack_300],
    trash_container=trash 
)

#####################################################################################################################################################
#####################################################################################################################################################

#Insert DES Generator here. 

reagents1 = convert_mole_fractions_to_volumes(3, 4, 0.3, 0.2, 0.8, 0.7, 96, 192 )

reagent_pos1 = ['A1', 'A3']



#####################################################################################################################################################
#####################################################################################################################################################
# STOCK LABWARE 1

robot.home()                                                    # Homes robot and prevents any pipette bugs 
for counter, reagent in enumerate(reagents1,0):
                                                                # These objects are temporary and will only exist within this loop
    source      = reagent_pos1[counter]                         # Counter is use to index an independent list (e.g. reagent_pos)
    P1000list   = [source]                                      # This is then added to both list - used in testing 				
    P300listn   = [source]


    P300.pick_up_tip()                                          # Picks up pipette tip for both P10 and P300 to allow to alternate
    P1000.pick_up_tip()
    for well_counter, values in enumerate(reagent):             # Specifies the well position and the volume of reagent being 
        if values == float(0):                                  # If volume is 0, well is skipped  
            pass
        elif values < float(300):
            P300.distribute(                                     # If volume below 300, P300 used not p1000. Greater than 300 P1000 is used.
                values, 
                Stock1(source), 
                wellplate_96(well_counter).top(0.5),          # Prevents submerging tip in solution, not completely sterile, but beneficial
                blow_out=True,                                  # Removes excess from tip
                rate=1,                                         # How quick it aspirates/dispenses, lower (ie 0.5) if stock viscous
                new_tip='never')
            P300.touch_tip(wellplate_96(well_counter))         # Touches tip to remove any droplets
            P300.blow_out(wellplate_96(well_counter))
        else: 
            P1000.distribute(
                values, 
                Stock1(source), 
                wellplate_96(well_counter).top(0.5),
                blow_out=True,
                rate=1,
                new_tip='never')
            P1000.touch_tip(wellplate_96(well_counter))
            P1000.blow_out(wellplate_96(well_counter))
    P1000.drop_tip()
    P300.drop_tip()

for c in robot.commands(): # remove this when uploading code to robot
     print(c)


 3840 feasible DES samples generated, clustered into 96 samples


/Users/studentuser/miniconda3/envs/opentrons/lib/python3.6/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Picking up tip well A1 in "1"
Picking up tip well A1 in "4"
Distributing 90.9826546042 from well A1 in "2" to well A1 in "3"
Transferring 90.9826546042 from well A1 in "2" to well A1 in "3"
Aspirating 90.9826546042 uL from well A1 in "2" at 1 speed
Dispensing 90.9826546042 uL into well A1 in "3"
Blowing out
Touching tip
Blowing out at well A1 in "3"
Distributing 62.46705526224137 from well A1 in "2" to well B1 in "3"
Transferring 62.46705526224137 from well A1 in "2" to well B1 in "3"
Aspirating 62.46705526224137 uL from well A1 in "2" at 1 speed
Dispensing 62.46705526224137 uL into well B1 in "3"
Blowing out
Touching tip
Blowing out at well B1 in "3"
Distributing 77.08267629301582 from well A1 in "2" to well C1 in "3"
Transferring 77.08267629301582 from well A1 in "2" to well C1 in "3"
Aspirating 77.08267629301582 uL from well A1 in "2" at 1 speed
Dispensing 77.08267629301582 uL into well C1 in "3"
Blowing out
Touching tip
Blowing out at well C1 in "3"
Distributing 99.02281483955413 f